### Custom JSON Encoding

#### Custom Encodings
As seen in the previous notes, any object can be serialized to JSON
- But this is cumbersome, we need to remember to call the JSON serializer for every class

And how do we do it for nested dictionaries and lists?

We use dump and dumps
- These can provide custom callable
- They use a default instance of hte JSONEncoder class
 - This means that we can completely override the JSONEncoder

#### Specifying a Custom Encoding Function

How do we specify a custom encoding function?
- One of the arguments of the dump / dumps function is default
 - When provided, Python will call default (or callable that we specify) if it encounters a type it cannot serialize
 - Therefore the argument(for default) must be a callable
 - And that callable must take a single argument
 - That argument will receive the object that Python cannot serialize
 - In the callable, we can include logic to differentiate betweeen different types
 - Or we can make use of a single dispatch generic function (using the @singledispatch decorator from the functools module)

#### Code Examples

In [1]:
from datetime import datetime

In [2]:
current = datetime.utcnow()

In [3]:
current

datetime.datetime(2020, 11, 16, 15, 52, 13, 777933)

In [4]:
import json

In [5]:
json.dumps(current)

TypeError: Object of type datetime is not JSON serializable

A typical datetime format is as follows:

YYYY-MM-DDTHH:MM:SS

There are variants like offsets (such as one for 7 hours 30 minutes):

YYYY-MM-DDTHH:MM:SS+07:30

In [6]:
str(current)

'2020-11-16 15:52:13.777933'

In [7]:
def format_iso(dt):
    return dt.strftime('%Y-%m-%dT%H:%M:%S')

In [8]:
format_iso(current)

'2020-11-16T15:52:13'

In [9]:
current.isoformat()

'2020-11-16T15:52:13.777933'

In [10]:
log_record = {'time': datetime.utcnow().isoformat(),
             'message': 'testing'}

In [11]:
print(json.dumps(log_record, indent=2))

{
  "time": "2020-11-16T15:59:57.727169",
  "message": "testing"
}


In [14]:
log_record = {'time': format_iso(datetime.utcnow()),
             'message': 'testing'}

print(json.dumps(log_record, indent=2))

{
  "time": "2020-11-16T16:01:06",
  "message": "testing"
}


In [15]:
log_record = {'time': datetime.utcnow(),
             'message': 'testing'}

In [16]:
json.dumps(log_record, default=format_iso)

'{"time": "2020-11-16T16:02:39", "message": "testing"}'

In [17]:
def format_general(arg):
    return 'Unknown serialization'

In [18]:
json.dumps(log_record, default=format_general)

'{"time": "Unknown serialization", "message": "testing"}'

In [20]:
log_record = {'time': format_iso(datetime.utcnow()),
             'message': 'testing',
             'args': {10, "test"}}

In [21]:
json.dumps(log_record, default=format_general)

'{"time": "2020-11-16T16:04:59", "message": "testing", "args": "Unknown serialization"}'

In [22]:
log_record = {
    'time1': datetime.utcnow(),
    'time2': datetime.utcnow(),
    'message': 'Testing...'
}

In [23]:
json.dumps(log_record, default=format_iso)

'{"time1": "2020-11-16T16:06:33", "time2": "2020-11-16T16:06:33", "message": "Testing..."}'

In [24]:
log_record = {
    'time1': datetime.utcnow(),
    'time2': datetime.utcnow(),
    'message': 'Testing...',
    'args': {1, 2, 3}
}

In [25]:
json.dumps(log_record, default=format_iso)

AttributeError: 'set' object has no attribute 'strftime'

In [28]:
def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    elif isinstance(arg, set):
        return list(arg)

In [31]:
json.dumps(log_record, default=custom_json_formatter)

'{"time1": "2020-11-16T16:08:07.251175", "time2": "2020-11-16T16:08:07.251175", "message": "Testing...", "args": [1, 2, 3]}'

In [35]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.create_dt = datetime.utcnow()
        
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def toJSON(self):
        return {
            'name': self.name,
            'age': self.age,
            'create_dt': self.create_dt.isoformat()
        }

In [38]:
p = Person('John', 82)

In [39]:
print(p)

Person(name=John, age=82)


In [42]:
p.toJSON()

{'name': 'John', 'age': 82, 'create_dt': '2020-11-16T16:13:50.360202'}

In [43]:
log_record = dict(time=datetime.utcnow(),
                  message='Created new person record.',
                  person=p
                 )

In [44]:
log_record

{'time': datetime.datetime(2020, 11, 16, 16, 16, 6, 229446),
 'message': 'Created new person record.',
 'person': Person(name=John, age=82)}

In [45]:
json.dumps(log_record, default=custom_json_formatter)

'{"time": "2020-11-16T16:16:06.229446", "message": "Created new person record.", "person": null}'

In [46]:
def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    elif isinstance(arg, set):
        return list(arg)
    elif isinstance(arg, Person):
        return arg.toJSON()

In [47]:
json.dumps(log_record, default=custom_json_formatter)

'{"time": "2020-11-16T16:16:06.229446", "message": "Created new person record.", "person": {"name": "John", "age": 82, "create_dt": "2020-11-16T16:13:50.360202"}}'

In [64]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.create_dt = datetime.utcnow()
        
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def toJSON(self):
        return {
            'name': self.name,
            'age': self.age,
            'create_dt': self.create_dt
        }

In [65]:
p = Person('Python', 45)

In [66]:
log_record = dict(time=datetime.utcnow(),
                  message='Created new person record.',
                  person=p
                 )

In [67]:
log_record

{'time': datetime.datetime(2020, 11, 16, 16, 21, 46, 433332),
 'message': 'Created new person record.',
 'person': Person(name=Python, age=45)}

In [68]:
json.dumps(log_record, default=custom_json_formatter)

'{"time": "2020-11-16T16:21:46.433332", "message": "Created new person record.", "person": {"name": "Python", "age": 45, "create_dt": "2020-11-16T16:21:46.288200"}}'

In [69]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.create_dt = datetime.utcnow()
        
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def toJSON(self):
        return vars(self)

In [70]:
p = Person('Python', 45)

In [71]:
log_record = dict(time=datetime.utcnow(),
                  message='Created new person record.',
                  person=p
                 )

In [72]:
log_record

{'time': datetime.datetime(2020, 11, 16, 16, 21, 47, 245069),
 'message': 'Created new person record.',
 'person': Person(name=Python, age=45)}

In [73]:
json.dumps(log_record, default=custom_json_formatter)

'{"time": "2020-11-16T16:21:47.245069", "message": "Created new person record.", "person": {"name": "Python", "age": 45, "create_dt": "2020-11-16T16:21:47.099937"}}'

In [74]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Point(x={self.x}, y={self.y})'

In [75]:
def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    elif isinstance(arg, set):
        return list(arg)
    else:
        try:
            return arg.toJSON()
        except AttributeError:
            try:
                return vars(arg)
            except TypeError:
                return str(arg)

In [77]:
from decimal import Decimal

pt1 = Point(1, 2)
p = Person('John', 18)
pt2 = Point(Decimal('10.5'), Decimal(100.5))

log_record = dict(time=datetime.utcnow(),
                  message='Created new point',
                  point=pt1,
                  point_2=pt2,
                  created_by=p
                 )

In [79]:
json.dumps(log_record, default=custom_json_formatter)

'{"time": "2020-11-16T16:30:58.923407", "message": "Created new point", "point": {"x": 1, "y": 2}, "point_2": {"x": "10.5", "y": "100.5"}, "created_by": {"name": "John", "age": 18, "create_dt": "2020-11-16T16:30:58.923407"}}'

In [80]:
from functools import singledispatch

In [83]:
@singledispatch
def json_format(arg):
    print(arg)
    try:
        print('\ttrying to use toJSON()...')
        return arg.toJSON()
    except AttributeError:
        print('\tfailed- trying to use var...')
        try:
            return vars(arg)
        except TypeError:
            print('\tfailed - using string repr...')
            return str(arg)

In [86]:
@json_format.register(datetime)
def _(arg):
    return arg.isoformat()

In [87]:
@json_format.register(set)
def _(arg):
    return list(arg)

In [88]:
log_record = dict(time=datetime.utcnow(),
                  message='Created new point',
                  point=pt1,
                  created_by=p
                 )

In [89]:
print(json.dumps(log_record, indent=2, default=json_format))

Point(x=1, y=2)
	trying to use toJSON()...
	failed- trying to use var...
Person(name=John, age=18)
	trying to use toJSON()...
{
  "time": "2020-11-16T16:37:26.649928",
  "message": "Created new point",
  "point": {
    "x": 1,
    "y": 2
  },
  "created_by": {
    "name": "John",
    "age": 18,
    "create_dt": "2020-11-16T16:30:58.923407"
  }
}


In [90]:
from decimal import Decimal
from fractions import Fraction

In [91]:
d = dict(a=1+1j,
        b=Decimal('0.5'),
        c=Fraction(1, 3),
        p=Person('Python', 27),
        pt=Point(0, 0),
         time=datetime.utcnow()
        )

In [92]:
d

{'a': (1+1j),
 'b': Decimal('0.5'),
 'c': Fraction(1, 3),
 'p': Person(name=Python, age=27),
 'pt': Point(x=0, y=0),
 'time': datetime.datetime(2020, 11, 16, 16, 40, 9, 2339)}

In [93]:
json.dumps(d,default=json_format)

(1+1j)
	trying to use toJSON()...
	failed- trying to use var...
	failed - using string repr...
0.5
	trying to use toJSON()...
	failed- trying to use var...
	failed - using string repr...
1/3
	trying to use toJSON()...
	failed- trying to use var...
	failed - using string repr...
Person(name=Python, age=27)
	trying to use toJSON()...
Point(x=0, y=0)
	trying to use toJSON()...
	failed- trying to use var...


'{"a": "(1+1j)", "b": "0.5", "c": "1/3", "p": {"name": "Python", "age": 27, "create_dt": "2020-11-16T16:40:09.002339"}, "pt": {"x": 0, "y": 0}, "time": "2020-11-16T16:40:09.002339"}'

In [95]:
@json_format.register(Decimal)
def _(arg):
    return f'Decimal({str(arg)})'

In [96]:
json.dumps(d,default=json_format)

(1+1j)
	trying to use toJSON()...
	failed- trying to use var...
	failed - using string repr...
1/3
	trying to use toJSON()...
	failed- trying to use var...
	failed - using string repr...
Person(name=Python, age=27)
	trying to use toJSON()...
Point(x=0, y=0)
	trying to use toJSON()...
	failed- trying to use var...


'{"a": "(1+1j)", "b": "Decimal(0.5)", "c": "1/3", "p": {"name": "Python", "age": 27, "create_dt": "2020-11-16T16:40:09.002339"}, "pt": {"x": 0, "y": 0}, "time": "2020-11-16T16:40:09.002339"}'